In [1]:
import os
import sys
import json
import time

In [2]:
# Disable GPUs
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Add current directory to path
if '.' not in sys.path:
  sys.path.insert(0, '.')

In [3]:
import tensorflow as tf

# Ignore warnings
tf.get_logger().setLevel('ERROR')

 create an mnist.py file with a simple model and dataset setup. 
 
 This python file will be used by the worker-processes in this tutorial.

The name of this file derives from the dataset you will be using which is called mnist and consists of 60,000 28x28 grayscale images of the first 10 digits.

In [6]:
# Import your mnist model
import mnist

# Set batch size
batch_size = 64

# Load the dataset
single_worker_dataset = mnist.mnist_dataset(batch_size)

# Load compiled CNN model
single_worker_model = mnist.build_and_compile_cnn_model()

# As training progresses, the loss should drop and the accuracy should increase.
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/3
70/70 [==============================] - 3s 37ms/step - loss: 2.2948 - accuracy: 0.1252
Epoch 2/3
70/70 [==============================] - 3s 37ms/step - loss: 2.2589 - accuracy: 0.1964
Epoch 3/3
70/70 [==============================] - 3s 38ms/step - loss: 2.2163 - accuracy: 0.3080


creating trainer-configuration for distributed trainer
TF_CONFIG with 2 workers on localhost
the task index to 0, this machine is the first worker and will be appointed as the chief worker.


In [7]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

In [8]:
json.dumps(tf_config)

'{"cluster": {"worker": ["localhost:12345", "localhost:23456"]}, "task": {"type": "worker", "index": 0}}'

Choosing training strategy 

In [9]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()

MultiWorkerMirroredStrategy creates copies of all variables in the model's layers on each device across all workers.

In [10]:
# Implementing distributed strategy via a context manager
with strategy.scope():
  multi_worker_model = mnist.build_and_compile_cnn_model()

In [13]:
# Set TF_CONFIG env variable
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [14]:
# first kill any previous runs
%killbgscripts

All background processes were killed.


In [15]:
%%bash --bg
python main.py &> job_0.log

In [16]:
# Wait for logs to be written to the file
time.sleep(10)

In [17]:
%%bash
cat job_0.log

2022-09-18 08:12:24.164257: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [18]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [19]:
%%bash
python main.py

Epoch 1/3
70/70 [==============================] - 10s 126ms/step - loss: 2.2770 - accuracy: 0.1927
Epoch 2/3
70/70 [==============================] - 9s 133ms/step - loss: 2.2255 - accuracy: 0.4388
Epoch 3/3
70/70 [==============================] - 9s 127ms/step - loss: 2.1622 - accuracy: 0.5886


2022-09-18 08:13:10.755039: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-18 08:13:11.738158: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_INT64
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 60000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 28
        }
        dim {
          size: 28
        }
      }
      shape {
      }
    

In [20]:
%%bash
cat job_0.log

2022-09-18 08:12:24.164257: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-18 08:13:11.724165: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_INT64
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 60000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 28
        }
        dim {
          size: 28
        }
      }
      shape {
      }
    